In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode

# ## Install NeMo
BRANCH = 'r1.0.0rc1'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[asr]

## Install TorchAudio
!pip install torchaudio>=0.6.0 -f https://download.pytorch.org/whl/torch_stable.html

## Grab the config we'll use in this example
!mkdir configs

# Introduction

Data augmentation is a useful method to improve the performance of models which is applicable across multiple domains. Certain augmentations can also substantially improve robustness of models to noisy samples. 

In this notebook, we describe how to construct an augmentation pipeline inside [Neural Modules (NeMo)](https://github.com/NVIDIA/NeMo), enable augmented training of a [MatchboxNet model](https://arxiv.org/abs/2004.08531 ) ( based on QuartzNet, from the paper ["QuartzNet: Deep Automatic Speech Recognition with 1D Time-Channel Separable Convolutions"](https://arxiv.org/abs/1910.10261)) and finally how to construct custom augmentations to add to NeMo.

The notebook will follow the steps below:

 - Dataset preparation: Preparing a noise dataset using an example file.

 - Construct a data augmentation pipeline.
 
 - Construct a custom augmentation and register it for use in NeMo.

## Note
Data augmentation is valuable for many datasets, but it comes at the cost of increased training time if samples are augmented during training time. Certain augmentations are particularly costly, in terms of how much time they take to process a single sample. A few examples of slow augmentations available in NeMo are : 

 - Speed Perturbation
 - Time Stretch Perturbation (Sample level)
 - Noise Perturbation
 - Impulse Perturbation
 - Time Stretch Augmentation (Batch level, Neural Module)
 
For such augmentations, it is advisable to pre-process the dataset offline for a one time preprocessing cost and then train the dataset on this augmented training set.

## Taking a Look at Our Data (AN4)

The AN4 dataset, also known as the Alphanumeric dataset, was collected and published by Carnegie Mellon University. It consists of recordings of people spelling out addresses, names, telephone numbers, etc., one letter or number at a time, as well as their corresponding transcripts. We choose to use AN4 for this tutorial because it is relatively small, with 948 training and 130 test utterances, and so it trains quickly.

Before we get started, let's download and prepare the dataset. The utterances are available as `.sph` files, so we will need to convert them to `.wav` for later processing. Please make sure you have [Sox](http://sox.sourceforge.net/) installed for this step (instructions to setup your environment are available at the beginning of this notebook).

In [ ]:
# This is where the an4/ directory will be placed.
# Change this if you don't want the data to be extracted in the current directory.
data_dir = '.'

In [ ]:
import glob
import os
import subprocess
import tarfile
import wget

# Download the dataset. This will take a few moments...
print("******")
if not os.path.exists(data_dir + '/an4_sphere.tar.gz'):
    an4_url = 'http://www.speech.cs.cmu.edu/databases/an4/an4_sphere.tar.gz'
    an4_path = wget.download(an4_url, data_dir)
    print(f"Dataset downloaded at: {an4_path}")
else:
    print("Tarfile already exists.")
    an4_path = data_dir + '/an4_sphere.tar.gz'

if not os.path.exists(data_dir + '/an4/'):
    # Untar and convert .sph to .wav (using sox)
    tar = tarfile.open(an4_path)
    tar.extractall(path=data_dir)

    print("Converting .sph to .wav...")
    sph_list = glob.glob(data_dir + '/an4/**/*.sph', recursive=True)
    for sph_path in sph_list:
        wav_path = sph_path[:-4] + '.wav'
        cmd = ["sox", sph_path, wav_path]
        subprocess.run(cmd)
print("Finished conversion.\n******")


You should now have a folder called `an4` that contains `etc/an4_train.transcription`, `etc/an4_test.transcription`, audio files in `wav/an4_clstk` and `wav/an4test_clstk`, along with some other files we will not need.

We now build a few manifest files which will be used later:

In [ ]:
# --- Building Manifest Files --- #
import json
import librosa

# Function to build a manifest
def build_manifest(transcripts_path, manifest_path, wav_path):
    with open(transcripts_path, 'r') as fin:
        with open(manifest_path, 'w') as fout:
            for line in fin:
                # Lines look like this:
                # <s> transcript </s> (fileID)
                transcript = line[: line.find('(')-1].lower()
                transcript = transcript.replace('<s>', '').replace('</s>', '')
                transcript = transcript.strip()

                file_id = line[line.find('(')+1 : -2]  # e.g. "cen4-fash-b"
                audio_path = os.path.join(
                    data_dir, wav_path,
                    file_id[file_id.find('-')+1 : file_id.rfind('-')],
                    file_id + '.wav')

                duration = librosa.core.get_duration(filename=audio_path)

                # Write the metadata to the manifest
                metadata = {
                    "audio_filepath": audio_path,
                    "duration": duration,
                    "text": transcript
                }
                json.dump(metadata, fout)
                fout.write('\n')
                
# Building Manifests
print("******")
train_transcripts = data_dir + '/an4/etc/an4_train.transcription'
train_manifest = data_dir + '/an4/train_manifest.json'
build_manifest(train_transcripts, train_manifest, 'an4/wav/an4_clstk')
print("Training manifest created.")

test_transcripts = data_dir + '/an4/etc/an4_test.transcription'
test_manifest = data_dir + '/an4/test_manifest.json'
build_manifest(test_transcripts, test_manifest, 'an4/wav/an4test_clstk')
print("Test manifest created.")
print("******")

## Prepare the path to manifest files

In [ ]:
dataset_basedir = os.path.join(data_dir, 'an4')

train_dataset = os.path.join(dataset_basedir, 'train_manifest.json')
test_dataset = os.path.join(dataset_basedir, 'test_manifest.json')

## Read a few rows of the manifest file 

Manifest files are the data structure used by NeMo to declare a few important details about the data :

1) `audio_filepath`: Refers to the path to the raw audio file <br>
2) `text`: The text transcript of this sample <br>
3) `duration`: The length of the audio file, in seconds.

In [ ]:
!head -n 5 {train_dataset}

# Data Augmentation Pipeline

Constructing a data augmentation pipeline in NeMo is as simple as composing a nested dictionary that describes two things :

1) The probability of that augmentation occurring - using the `prob` keyword <br>
2) The keyword arguments required by that augmentation class

Below, we show a few samples of these augmentations. Note, in order to distinguish between the original sample and the perturbed sample, we exaggerate the perturbation strength significantly.

In [ ]:
import torch
import IPython.display as ipd

## Audio file preparation 

In [ ]:
# Import the data augmentation component from ASR collection
from nemo.collections.asr.parts import perturb, segment

In [ ]:
# Lets see the available perturbations
perturb.perturbation_types

### Obtain a baseline audio file

In [ ]:
filepath = librosa.util.example_audio_file()
sample, sr = librosa.core.load(filepath)

ipd.Audio(sample, rate=sr)

### Convert to WAV format

In [ ]:
import soundfile as sf

# lets convert this ogg file into a wave to be compatible with NeMo
if not os.path.exists('./media'):
    os.makedirs('./media/')
    
filename = 'Kevin_MacLeod_-_Vibe_Ace.wav'
filepath = os.path.join('media', filename)

sf.write(filepath, sample, samplerate=sr)

In [ ]:
sample, sr = librosa.core.load(filepath)
ipd.Audio(sample, rate=sr)

In [ ]:
# NeMo has its own support class for loading wav files
def load_audio() -> segment.AudioSegment:
    filename = 'Kevin_MacLeod_-_Vibe_Ace.wav'
    filepath = os.path.join('media', filename)
    sample_segment = segment.AudioSegment.from_file(filepath, target_sr=sr)
    return sample_segment

sample_segment = load_audio()
ipd.Audio(sample_segment.samples, rate=sr)

## White Noise Perturbation

White Noise perturbation is performed by the following steps : <br>
1) Randomly sample the amplitude of the noise from a uniformly distributed range (defined in dB) <br>
2) Sample gaussian noise (mean = 0, std = 1) with same length as audio signal <br>
3) Scale this gaussian noise by the amplitude (in dB scale) <br>
4) Add this noise vector to the original sample

Notably, the original signal should not have a "hissing sound" constantly present in the perturbed version.

In [ ]:
white_noise = perturb.WhiteNoisePerturbation(min_level=-50, max_level=-30)

# Perturb the audio file
sample_segment = load_audio()
white_noise.perturb(sample_segment)

ipd.Audio(sample_segment.samples, rate=sr)

## Shift Perturbation

Shift perturbation is performed by the following steps : <br>
1) Randomly sample the shift factor of the signal from a uniformly distributed range (defined in milliseconds) <br>
2) Depending on the sign of the shift, we shift the original signal to the left or the right. <br>
3) The boundary locations are filled with zeros after the shift of the signal <br>

Notably, the perturbed signal below skips the first 25 to 50 seconds of the original audio below, and the remainder of the time is simply silence. 

In [ ]:
shift = perturb.ShiftPerturbation(min_shift_ms=25000.0, max_shift_ms=50000.0)

# Perturb the audio file 
sample_segment = load_audio()
shift.perturb(sample_segment)

ipd.Audio(sample_segment.samples, rate=sr)

## Data Dependent Perturbations

Some perturbations require an external data source in order to perturb the original sample. Noise Perturbation is a perfect example of one such augmentation that requires an external noise source dataset in order to perturb the original data.

### Preparing a manifest of "noise" samples

In [ ]:
# Lets prepare a manifest file using the baseline file itself, cut into 1 second segments

def write_manifest(filepath, data_dir='./media/', manifest_name='noise_manifest', duration_max=None, duration_stride=1.0, filter_long=False, duration_limit=10.0):
    if duration_max is None:
        duration_max = 1e9
                
    with open(os.path.join(data_dir, manifest_name + '.json'), 'w') as fout:
        
        try:
            x, _sr = librosa.load(filepath)
            duration = librosa.get_duration(x, sr=_sr)

        except Exception:
            print(f"\n>>>>>>>>> WARNING: Librosa failed to load file {filepath}. Skipping this file !\n")
            return

        if filter_long and duration > duration_limit:
            print(f"Skipping sound sample {filepath}, exceeds duration limit of {duration_limit}")
            return

        offsets = []
        durations = []

        if duration > duration_max:
            current_offset = 0.0

            while current_offset < duration:
                difference = duration - current_offset
                segment_duration = min(duration_max, difference)

                offsets.append(current_offset)
                durations.append(segment_duration)

                current_offset += duration_stride

        else:
            offsets.append(0.0)
            durations.append(duration)


        for duration, offset in zip(durations, offsets):
            metadata = {
                'audio_filepath': filepath,
                'duration': duration,
                'label': 'noise',
                'text': '_',  # for compatibility with ASRAudioText collection
                'offset': offset,
            }

            json.dump(metadata, fout)
            fout.write('\n')
            fout.flush()

        print(f"Wrote {len(durations)} segments for filename {filename}")
            
    print("Finished preparing manifest !")

In [ ]:
filename = 'Kevin_MacLeod_-_Vibe_Ace.wav'
filepath = os.path.join('media', filename)

# Write a "noise" manifest file
write_manifest(filepath, manifest_name='noise_1s', duration_max=1.0, duration_stride=1.0)

In [ ]:
# Lets read this noise manifest file
noise_manifest_path = os.path.join('media', 'noise_1s.json')

!head -n 5 {noise_manifest_path}

In [ ]:
# Lets create a helper method to load the first file in the train dataset of AN4
# Load the first sample in the manifest
def load_gsc_sample() -> segment.AudioSegment:
    with open(train_dataset, 'r') as f:
        line = f.readline()
        
    line = json.loads(line)
    gsc_filepath = line['audio_filepath']
    sample_segment = segment.AudioSegment.from_file(gsc_filepath)
    return sample_segment

gsc_sample_segment = load_gsc_sample()
ipd.Audio(gsc_sample_segment.samples, rate=16000)

## Noise Augmentation

Noise perturbation is performed by the following steps : <br>
1) Randomly sample the amplitude scale of the noise sample from a uniformly distributed range (defined in dB) <br>
2) Randomly choose an audio clip from the set of noise audio samples available <br>
3) Compute the gain (in dB) required for the noise clip as compared to the original sample and scale the noise by this factor <br>
4) If the noise snippet is of shorter duration than the original audio, then randomly select an index in time from the original sample, where the noise snippet will be added <br>
5) If instead the noise snippet is longer than the duration of the original audio, then randomly subsegment the noise snippet and add the full snippet to the original audio <br>

Notably, the noise perturbed sample should sound as if there are two sounds playing at the same time (overlapping audio) as compared to the original signal. The magnitude of the noise will be dependent on step (3) and the location where the noise is added will depend on steps (4) and (5).

In [ ]:
import random
rng = random.Random(0)
noise = perturb.NoisePerturbation(manifest_path=noise_manifest_path,
                                  min_snr_db=-10, max_snr_db=-10,
                                  max_gain_db=300.0, rng=rng)

# Perturb the audio file 
sample_segment = load_gsc_sample()
noise.perturb(sample_segment)

ipd.Audio(sample_segment.samples, rate=16000)

## RIR and Noise Perturbation
RIR augmentation with additive foreground and background noise.
In this implementation audio data is augmented by first convolving the audio with a Room Impulse Response
and then adding foreground noise and background noise at various SNRs. RIR, foreground and background noises
should either be supplied with a manifest file or as tarred audio files (faster).

### Prepare rir data and manifest

In [ ]:
# This is where the rir data will be downloaded.
# Change this if you don't want the data to be extracted in the current directory.
rir_data_path = '.'
!python ../../scripts/dataset_processing/get_openslr_rir_data.py --data_root {rir_data_path}
rir_manifest_path = os.path.join(rir_data_path, 'processed', 'rir.json')
!head -n 3 {rir_manifest_path}

### Create RIR instance

In [ ]:
rir = perturb.RirAndNoisePerturbation(rir_manifest_path=rir_manifest_path, 
                                      rir_prob=1,
                                      noise_manifest_paths=[noise_manifest_path], # use noise_manifest_path from previous step
                                      bg_noise_manifest_paths=[noise_manifest_path],
                                      min_snr_db=[20], # foreground noise snr
                                      max_snr_db=[20],
                                      bg_min_snr_db=[20], # background noise snr
                                      bg_max_snr_db=[20],
                                      noise_tar_filepaths=[None], # `[None]` to indicates that noise audio files are not tar.
                                      bg_noise_tar_filepaths=[None])

### Perturb the audio

In [ ]:
sample_segment = load_gsc_sample()
rir.perturb(sample_segment)
ipd.Audio(sample_segment.samples, rate=16000)

## Speed Perturbation

Speed perturbation changes the speed of the speech, but does not preserve pitch of the sound. Try a few random augmentations to see how the pitch changes with change in duration of the audio file.

**Note**: This is a very slow augmentation and is not advised to perform online augmentation for large datasets as it can dramatically increase training time.

In [ ]:
resample_type = 'kaiser_best'  # Can be ['kaiser_best', 'kaiser_fast', 'fft', 'scipy']
speed = perturb.SpeedPerturbation(sr, resample_type, min_speed_rate=0.5, max_speed_rate=2.0, num_rates=-1)

# Perturb the audio file 
sample_segment = load_gsc_sample()
speed.perturb(sample_segment)

ipd.Audio(sample_segment.samples, rate=16000)

## Time Stretch Perturbation

Time Stretch perturbation changes the speed of the speech, and also preserve pitch of the sound. 
Try a few random augmentations to see how the pitch remains close to the same with change in duration of the audio file.

### Note about speed optimizations

Time stretch is a costly augmentation, and can easily cause training time to increase drastically. It is suggested that one installs the `numba` library using conda to use a more optimized augmentation kernel.

```python
conda install numba
```

In [ ]:
time_stretch = perturb.TimeStretchPerturbation(min_speed_rate=0.5, max_speed_rate=2.0, num_rates=3)

# Perturb the audio file 
sample_segment = load_gsc_sample()
time_stretch.perturb(sample_segment)

ipd.Audio(sample_segment.samples, rate=16000)

# Augmentation Pipeline

The augmentation pipeline can be constructed in multiple ways, either explicitly by instantiating the objects of these perturbations or implicitly by providing the arguments to these augmentations as a nested dictionary.

We will show both approaches in the following sections

## Explicit definition

### Instantiate the perturbations

In [ ]:
perturbations = [
    perturb.WhiteNoisePerturbation(min_level=-90, max_level=-46),
    perturb.GainPerturbation(min_gain_dbfs=0, max_gain_dbfs=50),
    perturb.NoisePerturbation(manifest_path=noise_manifest_path,
                              min_snr_db=0, max_snr_db=50, max_gain_db=300.0)
]

### Select chance of perturbations being applied

In [ ]:
probas = [1.0, 1.0, 0.5]

### Prepare the audio augmentation object

In [ ]:
augmentations = list(zip(probas, perturbations))

audio_augmentations = perturb.AudioAugmentor(augmentations)
audio_augmentations._pipeline

## Implicit definition

Implicit definitions are preferred since they can be prepared in the actual configuration object.

In [ ]:
perturb.perturbation_types  # Available perturbations

### Prepare the nested dictionary

In [ ]:
audio_augmentations = dict(
    white_noise = dict(
        prob=1.0,
        min_level=-90,
        max_level=-46
    ),
    gain = dict(
        prob=1.0,
        min_gain_dbfs=0,
        max_gain_dbfs=50
    ),
    noise = dict(
        prob=0.5,
        manifest_path=noise_manifest_path,
        min_snr_db=0,
        max_snr_db=50,
        max_gain_db=300.0
    )
)

audio_augmentations

### Supply `augmentor` as an argument to the `model.train_ds` config

Most of the common datasets used by ASR models support the keyword `augmentor` - which can include a nested dictionary defining the implicit definition of an augmentation pipeline.

Note, all ASR models support implicit declaration of augmentations. This includes - 

1) Speech To Label Models <br>
2) Speech To Text Models <br>
3) Speech To Text Models with BPE/WPE Support<br>

# Training - Application of augmentations

We will be describing the data loaders for a MatchboxNet model from the paper "[MatchboxNet: 1D Time-Channel Separable Convolutional Neural Network Architecture for Speech Commands Recognition](https://arxiv.org/abs/2004.08531)". The benefit of MatchboxNet over JASPER models is that they use Separable Convolutions, which greatly reduce the number of parameters required to get good model accuracy.

<ins>Care must be taken not to apply augmentations to the test set!</ins>


In [ ]:
from omegaconf import OmegaConf

In [ ]:
# We will download the MatchboxNet configuration file for either v1 or v2 dataset here
DATASET_VER = 1

if DATASET_VER == 1:
  MODEL_CONFIG = "matchboxnet_3x1x64_v1.yaml"
else:
  MODEL_CONFIG = "matchboxnet_3x1x64_v2.yaml"

if not os.path.exists(f"configs/{MODEL_CONFIG}"):
  !wget -P configs/ "https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/{MODEL_CONFIG}"

In [ ]:
# This line will load the entire config of the MatchboxNet model
config_path = f"configs/{MODEL_CONFIG}"
config = OmegaConf.load(config_path)

### Augmentation in train set only

Note how the train dataset config supports the `augmentor` implicit definition, however the test config does not.

This is essential to avoid mistakenly performing Test Time Augmentation.

In [ ]:
# Has `augmentor`
print(config.model.train_ds.pretty())

In [ ]:
# Does not have `augmentor`
print(config.model.test_ds.pretty())

# Custom Perturbations

We can define and use custom perturbations as required simply by extending the `Perturbation` class. 

Let's look at how we can build a custom Noise Perturbation that we can use to evaluate the effect of noise at inference time, in order to measure the model's robustness to noise

In evaluation mode, we want to set an explicit value for the `snr_db` parameter instead of uniformly sample it from a range. This allows us to control the signal to noise ratio without relying on randomness from the training implementation of `NoisePerturbation`.

Further, we force a random seed in order to produce reproduceable results on the evaluation set.

With this combination, we can easily evaluate each sample in the test set `S` times (`S` being the number of random seeds), and can evaluate each of these samples at `D` levels of Signal to Noise Ratio (in dB). 

In [ ]:
# We use a NeMo utility to parse the manifest file for us
from nemo.collections.asr.parts import collections, parsers

class NoisePerturbationEval(perturb.Perturbation):
    def __init__(
        self, manifest_path=None, snr_db=40, max_gain_db=300.0, seed=None,
    ):
        seed = seed if seed is not None else 0
        self._manifest = collections.ASRAudioText(manifest_path, parser=parsers.make_parser([]))
        self._snr_db = snr_db
        self._max_gain_db = max_gain_db
        self._rng = random.Random(seed)
    
    # This is mostly obtained from the original NoisePerturbation class itself
    def perturb(self, data):
        snr_db = self._snr_db
        noise_record = self._rng.sample(self._manifest.data, 1)[0]
        noise = AudioSegment.from_file(noise_record.audio_file, target_sr=data.sample_rate)
        noise_gain_db = min(data.rms_db - noise.rms_db - snr_db, self._max_gain_db)

        # calculate noise segment to use
        start_time = 0.0
        if noise.duration > (start_time + data.duration):
            noise.subsegment(start_time=start_time, end_time=start_time + data.duration)

        # adjust gain for snr purposes and superimpose
        noise.gain_db(noise_gain_db)

        if noise._samples.shape[0] < data._samples.shape[0]:
            noise_idx = data._samples.shape[0] // 2  # midpoint of audio
            while (noise_idx + noise._samples.shape[0]) > data._samples.shape[0]:
                noise_idx = noise_idx // 2  # half the initial starting point

            data._samples[noise_idx: noise_idx + noise._samples.shape[0]] += noise._samples

        else:
            data._samples += noise._samples


## Registering augmentations

We can use either approach to submit this test time augmentation to the Data Loaders.

In order to obtain the convenience of the implicit method, we must register this augmentation into NeMo's directory of available augmentations. This can be done as follows -

In [ ]:
perturb.register_perturbation(name='noise_eval', perturbation=NoisePerturbationEval)

In [ ]:
# Lets check the registry of allowed perturbations !
perturb.perturbation_types

## Overriding pre-existing augmentations

**Note**: It is not allowed to overwrite already registered perturbations using the `perturb.register_perturbation` method. It will raise a `ValueError` in order to prevent overwriting the pre-existing perturbation types